In [1]:
%run base.ipynb

# EPIC 1 - User Story DF-24
## Erstellung eines Notebooks und aufrufen der Daten per Dataland-API

Als QA-Lab-Team möchten wir ein Notebook erstellen, mit dem wir eine Anfrage an die Dataland-API senden, sodass wir anhand der Berichts-ID den Unternehmensbericht erhalten.

Beschreibung:<br>
Nutzen von Dataland Endpoint /data/nuclear-and-gas/companies/{companyId} im Nuclear & Gas Controller. Die Funktionalität wird außerhalb des Notebooks implementiert.

Akzeptanzkriterien:<br>
    - Notebook ist auf jedem Laptop ausführbar<br>
    - Anfrage an Dataland gibt Daten zurück

## 1.Schritt: Dataset mit Hilfe der Company_ID über die Dataland_API aufrufen
Mithilfe der Company_ID wird die Dataland_API aufgerufen. Nachdem Eintragen aus welcher Periode das Dataset sein soll, ist es möglich den Wert 1 zu erlangen.

In [3]:
import dataland_qa_lab.dataland.get_data as qa

company_id = "9f6f6cea-7316-4101-aba9-170023c811e8"
year = "2020"

all_datasets = qa._get_all_company_datasets_(company_id=company_id)

data_id = qa._get_data_id_by_year_(company_id=company_id, year=year)
print(data_id)

dataset_by_year = qa._get_dataset_by_year_(company_id=company_id, year=year)

value1 = qa._get_value1_by_year_(company_id=company_id, year=year)
print(value1)

datasource_reference = qa._get_datasource_reference_(company_id=company_id, year=year)
print(datasource_reference)

b7ada81e-3fa2-4070-a136-8e247acb486f
No
b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n<</Type/Catalog/Pages 2 0 R/Lang(de-DE) /StructTreeRoot 10 0 R/MarkInfo<</Marked true>>/Metadata 20 0 R/ViewerPreferences 21 0 R>>\r\nendobj\r\n2 0 obj\r\n<</Type/Pages/Count 1/Kids[ 3 0 R] >>\r\nendobj\r\n3 0 obj\r\n<</Type/Page/Parent 2 0 R/Resources<</Font<</F1 5 0 R>>/ExtGState<</GS7 7 0 R/GS8 8 0 R>>/ProcSet[/PDF/Text/ImageB/ImageC/ImageI] >>/MediaBox[ 0 0 595.44 842.04] /Contents 4 0 R/Group<</Type/Group/S/Transparency/CS/DeviceRGB>>/Tabs/S/StructParents 0>>\r\nendobj\r\n4 0 obj\r\n<</Filter/FlateDecode/Length 179>>\r\nstream\r\nx\x9c\xad\x8e=\x0b\xc20\x18\x84\xf7@\xfe\xc3\x8d\x89`\x9a\xd7$M\n\xa5\x83m-\n\x05\xc5\x80\x83\xb8Y\x9d\x14\xbf\xfe?\xd6\xd2Awo{\xe0\xb8{\x90\xac\x91\xe7I[.+\xe8\xa2\xc0\xbc*q\xe7L+\xfdI\x08\xde@\xc3eNY\x8b`gJ[<:\xcev\x13\\9\x9bG\xce\x92\x05\x81HQ\x8ax\xe2\x8c\xfa\xb6\x06\xc1k\x95\xa5\x01\xdee\xca9\xc4K\xdfk\xb6\x1e\xe7g?\x8d\xf3@a\xa4\x86\xb3\xbd\x88\x92\xb4\xe8\x9erJN\x

## 2.Schritt: Laden aller Daten
Hier werden die Daten von Dataland geladen und alle 6 Werte gesondert gespeichert.

from dataland_qa_lab.utils import config

conf = config.get_config()
dataland_client = conf.dataland_client

api = dataland_client.eu_taxonomy_nuclear_and_gas_api
dataset = api.get_all_company_nuclear_and_gas_data(company_id=company_id)

# Eintragen aus welcher Periode man das Dataset haben will -> In diesem Fall 2024
data_id = "test"
for t in range(len(dataset)):
    if (dataset[t].meta_info.reporting_period == year):
        data_id = dataset[t].meta_info.data_id
        break

data = dataland_client.eu_taxonomy_nuclear_and_gas_api.get_company_associated_nuclear_and_gas_data(data_id=data_id)

wert1 = data.data.general.general.nuclear_energy_related_activities_section426
wert2 = data.data.general.general.nuclear_energy_related_activities_section427
wert3 = data.data.general.general.nuclear_energy_related_activities_section428
wert4 = data.data.general.general.fossil_gas_related_activities_section429
wert5 = data.data.general.general.fossil_gas_related_activities_section430
wert6 = data.data.general.general.fossil_gas_related_activities_section431
print(data)
print(wert1)
print(wert2)
print(wert3)
print(wert4)
print(wert5)
print(wert6)

## 3.Schritt: Datenquelle laden
In diesem Fall wird die Dateireferenz von wert1 genutzt um das dazugehörige Dokument zu bekommen.

## 4.Schritt: Daten aus dem Dokument extrahieren und in Text umwandeln

import io

import pypdf

full_document_byte_stream = io.BytesIO(document_bytes)
full_pdf = pypdf.PdfReader(full_document_byte_stream)

partial_document_byte_stream = io.BytesIO()
partial_pdf = pypdf.PdfWriter()

partial_pdf.add_page(full_pdf.get_page(int(wert1.data_source.page) - 1))  # Correct for 0 offset
partial_pdf.write(partial_document_byte_stream)
partial_document_byte_stream.seek(0)
None

from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from azure.core.credentials import AzureKeyCredential

docintel_cred = AzureKeyCredential(conf.azure_docintel_api_key)
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=conf.azure_docintel_endpoint, credential=docintel_cred
)

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout",
    analyze_request=partial_document_byte_stream,
    content_type="application/octet-stream",
    output_content_format=ContentFormat.MARKDOWN,
)
result: AnalyzeResult = poller.result()

#### Das Ergebnis wird als Markdown direkt im Notebook wiedergegeben

from IPython.display import Markdown, display

display(Markdown(result.content))